<a href="https://colab.research.google.com/github/dasharakelova/essay_grading/blob/main/%22word2vec%2Blinear_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install razdel

In [3]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=dd88a72902560dfd371b9acd14648f919d542379b1295306f919d334f8eeb6e8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [1]:
import gensim.downloader
import pymorphy2

from razdel import tokenize
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import nltk
import random
import string

import bs4 as bs
import urllib.request
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

Токенизация

In [6]:
def razdel_tokenize(text):
    tokens = []
    tokens_raw = list(tokenize(text))
    for token in tokens_raw:
        tokens.append(token.text)

    return tokens

In [7]:
razdel_tokenize('Я царь Шумера и Аккада')

['Я', 'царь', 'Шумера', 'и', 'Аккада']

In [8]:
import json
with open('output.json') as cat_file:
    data = json.load(cat_file)

In [22]:
X_train = []
y_train = []
for key, value in data.items():
    X_train.append(razdel_tokenize(value['text']))
    y_train.append(value['score'])

print(X_train[2:5])

[['Как', 'сохранить', 'в', 'своей', 'душе', 'доброту', 'в', 'ситуации', ',', 'когда', 'мы', 'испытываем', 'жестокие', 'удары', 'судьбы', '?', 'Можно', 'ли', 'оставаться', 'человечным', ',', 'если', 'вокруг', 'много', 'зла', 'и', 'несправедливости', '?', 'Правда', 'ли', ',', 'что', 'именно', 'из', 'детства', 'мы', 'черпаем', 'запасы', 'доброты', 'и', 'любви', '?', 'Именно', 'эти', 'вопросы', 'возникают', 'при', 'чтении', 'текста', 'Даниила', 'Гранина', '.', 'Раскрывая', 'проблему', 'сохранения', 'человечности', 'в', 'тяжёлых', 'жизненных', 'условиях', ',', 'автор', 'опирается', 'на', 'собственные', 'рассуждения', '.', 'Он', 'обращается', 'к', 'речи', 'героя', 'Ф', '.', 'М', '.', 'Достоевского', 'Алёши', 'Карамазова', ',', 'акцентируя', 'наше', 'внимание', 'на', 'том', ',', 'что', 'нет', 'ничего', 'выше', 'и', 'полезнее', ',', 'чем', 'хорошее', 'воспоминание', ',', 'вынесенное', 'из', 'детства', ',', 'из', 'родительского', 'дома', '.', 'Алёша', 'Карамазов', ',', 'а', 'вместе', 'с', 'ним'

Лемматизация

In [2]:
morpher = pymorphy2.MorphAnalyzer()

In [57]:
token = 'конфету'
parsed = morpher.parse(token)[0]

In [58]:
lemma, pos = parsed.normal_form, parsed.tag.POS

In [59]:
if pos in ['NOUN', 'INFN', 'VERB', 'ADJF']:
    if pos == 'INFN':
        pos = 'VERB'
    if pos == 'ADJF':
        pos = 'ADJ'

In [73]:
print(lemma)
print(pos)

,_none
None


In [4]:
b = np.array([1.7, 3, 6.8678, 3])
a = np.array([0 for i in range(len(b))])
a = a + b
print(c)

[1.7    3.     6.8678 3.    ]


Word2vec

In [3]:
w2v = gensim.downloader.load('word2vec-ruscorpora-300')

In [11]:
vec = np.array([])
x_train = []
for i in range(len(X_train)):
    vec = np.array([0 for o in range(300)])
    for j in range(len(X_train[i])):
        token = X_train[i][j]
        # print(token)
        parsed = morpher.parse(token)[0]
        lemma, pos = parsed.normal_form, parsed.tag.POS
        # print(type(pos))
        if pos in ['NOUN', 'INFN', 'VERB', 'ADJF']:
            if pos == 'INFN':
                pos = 'VERB'
            if pos == 'ADJF':
                pos = 'ADJ'
        try:
          vec = vec + w2v[f'{lemma}_{pos}']
          # vec = vec/len(X_train[i])

        except:
          ...
    x_train.append(vec)
print(len(x_train))
print(x_train[:3])

2701
[array([-3.14246746e-01, -7.86873254e-01,  1.78376903e+00,  7.46931090e+00,
       -2.73300441e+00,  4.80715453e+00,  1.65844414e+00,  4.30652788e+00,
        1.78259164e+00, -5.79683713e+00,  4.12086772e+00, -1.02186512e+01,
       -5.95003852e+00, -2.31250598e+00,  5.83560196e+00,  3.85951009e-01,
        6.10110054e+00,  2.70206956e+00,  3.48409300e+00,  3.50005174e+00,
        6.75213879e+00,  7.18707309e+00, -1.16845743e+01,  7.16183376e+00,
        1.51956063e+00, -3.95898579e+00,  3.81932284e+00, -2.10941096e+00,
       -1.44835085e+00,  1.08005991e+00,  7.82637637e+00,  6.07788484e+00,
        9.29860880e+00, -6.84645607e+00,  7.71954346e+00,  3.19260813e+00,
        3.93723365e+00,  1.69305131e+00, -3.84464392e+00,  1.87134447e+00,
        4.49266918e+00,  2.77104806e+00,  3.26012973e+00, -8.46900237e-01,
       -3.24426758e+00,  4.21486673e+00,  4.44108158e+00, -9.54254513e-01,
        6.25130859e+00, -5.77535377e+00, -1.95317921e+00,  1.25289317e+00,
        8.74389022e

In [102]:
print(x_train[2:5])

[array([], dtype=float64), array([], dtype=float64), array([], dtype=float64)]


In [21]:
print(len(y_train))

1501


In [23]:
x_test = x_train[-600:]
x_train = x_train[:-600]
y_test = y_train[-600:]
y_train = y_train[:-600]

In [25]:
print(len(y_train))
print(len(x_train))

2101
2101


In [28]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(x_test)
# print(y_pred)

In [29]:
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-0.08655465 -0.13451832 -0.06331743  0.0263623  -0.10877552 -0.00823645
  0.0137895   0.103548   -0.0652053   0.05769369  0.14983014  0.04738231
  0.05430079 -0.00456684 -0.08251783  0.01782126 -0.03028122 -0.02147302
  0.09198788  0.00067916  0.09902293 -0.14790466 -0.12861158  0.02001151
  0.12399451 -0.10696039  0.05563475 -0.03523725 -0.11821185 -0.14690331
  0.00544198 -0.18861656 -0.01088183  0.03528598 -0.04272703 -0.03754157
 -0.07814662 -0.00264961  0.08859749  0.04939624 -0.05602684 -0.0137861
  0.0739239  -0.00217189  0.12010623 -0.0043122  -0.11302723 -0.04475827
 -0.1171398  -0.12886577  0.07327414 -0.02421441 -0.01429534  0.08508263
  0.11558142 -0.11578005  0.01790145  0.14336424 -0.00958571  0.04494178
 -0.05443826  0.03129812  0.06198907  0.0492729   0.17029949 -0.03646619
 -0.10634532 -0.05007102  0.03492504 -0.01742878  0.13262795  0.05038739
  0.01198027 -0.10458769 -0.00167514  0.10491892  0.16818483  0.01754957
 -0.08185149  0.08845628  0.1177255 